In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import scipy
import numba as nb
import time
from numba import guvectorize, vectorize, float64, int64, void

In [15]:
NUM_CHLD = 1000000
WISH_SIZE = 100
PREF_SIZE = 1000
NUM_GIFT = 1000
NUM_TRIP = 1667
NUM_TWIN = 20000
NUM_SING = NUM_CHLD - 2*NUM_TWIN - 3*NUM_TRIP
NUM_UNIT = NUM_SING + NUM_TWIN + NUM_TRIP
HIDDEN = 1024
print(NUM_SING)
print(NUM_UNIT)

954999
976666


In [3]:
def load_data(data_path):
    return pd.read_csv(data_path, header=None).as_matrix()[:, 1:]
%time chld_wish_list = load_data('../data/child_wishlist_v2.csv')
%time gift_pref_list = load_data('../data/gift_goodkids_v2.csv')
print(chld_wish_list.shape)
print(gift_pref_list.shape)
%time print(chld_wish_list.max())
%time print(gift_pref_list.max())

CPU times: user 9.92 s, sys: 1.37 s, total: 11.3 s
Wall time: 11.3 s
CPU times: user 230 ms, sys: 11.2 ms, total: 241 ms
Wall time: 233 ms
(1000000, 100)
(1000, 1000)
999
CPU times: user 123 ms, sys: 290 µs, total: 123 ms
Wall time: 123 ms
999999
CPU times: user 1.43 ms, sys: 96 µs, total: 1.53 ms
Wall time: 1.3 ms


In [4]:
## Creating C, We need to treat singles, twins, triplets separately
def get_wish_score(wish):
    assert(wish.max() < NUM_GIFT)
    assert(wish.shape[0] == NUM_CHLD)
    ans = np.zeros((NUM_CHLD, NUM_GIFT)) - 1.
    for i in range(wish.shape[0]):
        for j in range(wish.shape[1]):
            ans[i][wish[i][j]] = WISH_SIZE - j
    ans /= WISH_SIZE
    return ans
%time C = get_wish_score(chld_wish_list)
for i in range(NUM_TRIP):
    C[i] = C[3*i] + C[3*i+1] + C[3*i+2]
for i in range(NUM_TWIN):
    C[3*NUM_TRIP + i] = C[3*NUM_TRIP + 2*i] + C[3*NUM_TRIP + 2*i + 1]
C = np.concatenate([C[:NUM_TRIP], C[3*NUM_TRIP: 3*NUM_TRIP + NUM_TWIN], C[3*NUM_TRIP + 2*NUM_TWIN:]])
print(C.shape)

CPU times: user 1min 10s, sys: 15.4 s, total: 1min 26s
Wall time: 1min 25s
(976666, 1000)


In [6]:
## Getting pref_score
def get_pref_score(pref):
    assert(pref.max() < NUM_CHLD)
    assert(pref.shape[0] == NUM_GIFT)
    ans = np.zeros((NUM_CHLD, NUM_GIFT)) - 1.
    for i in range(pref.shape[0]):
        for j in range(pref.shape[1]):
            ans[pref[i][j]][i] = PREF_SIZE - j
    ans /= PREF_SIZE
    return ans
%time S = get_pref_score(gift_pref_list)
for i in range(NUM_TRIP):
    S[i] = S[3*i] + S[3*i+1] + S[3*i+2]
for i in range(NUM_TWIN):
    S[3*NUM_TRIP + i] = S[3*NUM_TRIP + 2*i] + S[3*NUM_TRIP + 2*i + 1]
S = np.concatenate([S[:NUM_TRIP], S[3*NUM_TRIP: 3*NUM_TRIP + NUM_TWIN], S[3*NUM_TRIP + 2*NUM_TWIN:]])
print(S.shape)

CPU times: user 5.03 s, sys: 3.57 s, total: 8.6 s
Wall time: 8.6 s
(976666, 1000)


In [14]:
x_c = tf.placeholder(tf.float32, [None, NUM_GIFT])
x_s = tf.placeholder(tf.float32, [None, NUM_GIFT])

In [18]:
wc = tf.Variable(tf.random_normal((NUM_GIFT, HIDDEN)))
bc = tf.Variable(tf.random_normal([HIDDEN]))
ws = tf.Variable(tf.random_normal((NUM_GIFT, HIDDEN)))
bs = tf.Variable(tf.random_normal([HIDDEN]))
layer_c = tf.nn.relu(tf.matmul(x_c, wc) + bc)
layer_s = tf.nn.relu(tf.matmul(x_c, wc) + bs)

In [23]:
hidden = tf.concat([layer_c, layer_s], 1)
wh = tf.Variable(tf.random_normal((2 * HIDDEN, NUM_GIFT)))
bh = tf.Variable(tf.random_normal([NUM_GIFT]))
y = tf.nn.softmax(tf.matmul(hidden, wh) + bh)

In [24]:
obj_c = tf.reduce_mean(tf.matmul(x_c, y))
obj_s = tf.reduce_mean(tf.matmul(x_s, y))
obj = -obj_c**3 - obj_s**3

In [27]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.005).minimize(obj)

In [28]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [38]:
def next_batch(num, c, s):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 ,len(c))
    np.random.shuffle(idx)
    idx = idx[:num]
    c_shuffle = [c[i] for i in idx]
    s_shuffle = [s[ i] for i in idx]

    return np.asarray(c_shuffle), np.asarray(s_shuffle)

In [43]:
total_iteration = 0
def optimize(C_input, S_input, num_iterations = 500, batch_size = 256):
    start_time = time.time()
    for i in range(num_iterations):

        # Get training set
        c_batch, s_batch = next_batch(batch_size, C_input, S_input)
        train_set = {
            x_c: c_batch,
            x_s: s_batch,
        }
        sess.run(optimizer, feed_dict = train_set)
        if self.total_iteration%50 == 0:
            objc = sess.run(obj, feed_dict = train_set)

            # Message for printing
            msg = "Optimization Iteration: {0:>6}, objective: {1:>6.1%}"
            print(msg.format(total_iteration, objc))
            total_iteration += 1
    print("One set of training: DONE!")

    end_time = time.time()
    time_dif = end_time - start_time
    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))
    return None

In [44]:
optimize(C, S)

InvalidArgumentError: Matrix size-incompatible: In[0]: [256,1000], In[1]: [256,1000]
	 [[Node: MatMul_12 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_Placeholder_3_0_0/_9, Softmax_1)]]

Caused by op u'MatMul_12', defined at:
  File "/opt/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/opt/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/opt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/opt/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
   